This is a eMail Spam Classifers that uses Naive Bayes supervised machine learning algorithm.

In [13]:
import os
import numpy as np
from collections import Counter
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score
from google.colab import drive
#import all useful packages here 
drive.mount('/content/drive/')
#import packages and mounted to read files from dirve

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


This function builds a Dictionary of most common 3000 words from all the email content. First it adds all words and symbols in the dictionary. Then it removes all non-alpha-numeric characters and any single character alpha-numeric characters. After this is complete it shrinks the Dictionary by keeping only most common 3000 words in the dictionary. It returns the Dictionary.

In [14]:
#clean up the data
def make_Dictionary(root_dir):              #create a function called make_Dictionary
  all_words = []                            #create a new empty list called all_words
  emails = [os.path.join(root_dir,f) for f in os.listdir(root_dir)] 
  #create a new list called emails, it join root_dir and f using os.path.join, f is in the list of root_dir
  #os.path.join() means join one or more path together
  #os.listdir() will print out all files in the path 
  for mail in emails: 
    with open(mail) as m:                   #reading mail in the emails list, open each mail and call it m 
      for line in m:
        words = line.split()                #Then split each word in line in the mail and call the list words
        all_words += words                  #Then add words into all_words
  dictionary = Counter(all_words)           #Counter is a dictionary subclass that counting hashable object, so it will count frequency in all_words and store it to dictionary
  list_to_remove = list(dictionary)         #make a list of dictionary and call the list list_to_remove

  for item in list_to_remove:               #for any item in the list list_to_remove
    if item.isalpha() == False:             #if the item is not equal to alpha
      del dictionary[item]                  #then delete it from the dictionary item
    elif len(item) == 1:                    #else, if length of the item is equal to one
      del dictionary[item]                  #also delete it
  dictionary = dictionary.most_common(3000) #using most_common return a list of most common 3000 words in the dictionary and store it back to dictionary
  return dictionary                         #output is the dictionary, and will delete words that are useless for analyze like 'i, he, are'
            

This function extracts feature columns and populates their values (Feature Matrix of 3000 comumns and rows equal to the number of email files). The function also analyzes the File Names of each email file and decides if it's a Spam or not based on the naming convention. Based on this the function also creates the Labelled Data Column. This function is used to extract the training dataset as well as the testing dataset and returns the Feature Dataset and the Label column.

In [15]:
#feature extraction
def extract_features(mail_dir):                                   #create a function called extract_features
  files = [os.path.join(mail_dir,fi) for fi in os.listdir(mail_dir)]   #create a new list called files, it join mail_dir and fi using os.path.join, fi is in the list of root_dir
  features_matrix = np.zeros((len(files),3000))                   #use np.zeros to return an array of zeros with rows (equal to the length of files) and 3000 columns and store the list to feature_matrix
  train_labels = np.zeros(len(files))                             #create an new array of zeros, only contain columns in the list and column is equal to the length of files and store the list to train_labels
  count = 1;                                                      #start counting from 1
  docID = 0;                                                      #docID starts from 0
  for fil in files: 
    with open(fil) as fi:                                         #reading fil in the files list first, open each fil and call it fi 
      for i, line in enumerate(fi):                               #use enumerate() will give you the count and value, so count fi
        if i ==2:                                                 #if i is equal to 2 which means body of email is at the third line of the text file
          words = line.split()                                    #then split the line and store the list to words
          for word in words:                                      #each word in list words, it starts from 0
            wordID = 0
            for i, d in enumerate(dictionary):                    #reading i, d from dictionary
              if d[0] == word:  
                wordID = i                                        #if the first item in d is word, then wordID is equal to 1 
                features_matrix[docID,wordID] = words.count(word) #put docID and wordID back into feature_matrix and count word
      train_labels[docID] = 0;
      filepathTokens = fil.split('/')                             #a new list filepathTokens is to split fil with '/'
      lastToken = filepathTokens[len(filepathTokens)-1]           #lastToken contains less 1 than filepathTokens
      if lastToken.startswith("spmsg"):
        train_labels[docID] = 1; 
        count = count + 1
      docID = docID + 1                                           #if the lastToken start with 'spmsg', docID for train_labels stars at 1 and add 1 each time
  return features_matrix, train_labels                            #the output is feature_matrix and train labels, create a dictionary with words and frequency, each row represents one message and column is word frequency
                                                                  #in order to check if the occurence of this word is in the dictionary           

The section is the main Program that calls the above two functions and gets executed first. First it "trains" the model using model.fit function and Training Dataset. After that it scores the Test Data set by running the Trained Model with the Test Data set. At the end it prints the model performance in terms of accuracy score.

In [5]:
TRAIN_DIR = '/content/drive/My Drive/MSBA_Colab_2020/ML Algorithms/Naive_Bayes/train-mails'
TEST_DIR = '/content/drive/My Drive/MSBA_Colab_2020/ML Algorithms/Naive_Bayes/test-mails'
#read files from google drive

In [12]:
dictionary = make_Dictionary(TRAIN_DIR)

print ("reading and processing emails from TRAIN and TEST folders")
features_matrix, labels = extract_features(TRAIN_DIR)
test_features_matrix, test_labels = extract_features(TEST_DIR)

model = GaussianNB() #we are using Gaussian Naive Bayes model here

print ("Training Model using Gaussian Naibe Bayes algorithm .....")
model.fit(features_matrix, labels) #use the Gaussian model to fit feature_matrix
print ("Training completed")
print ("testing trained model to predict Test Data labels")
predicted_labels = model.predict(test_features_matrix) #use Gaussian model to predict test_features_matrix
print ("Completed classification of the Test Data .... now printing Accuracy Score by comparing the Predicted Labels with the Test Labels:")
print (accuracy_score(test_labels, predicted_labels))
#the accuracy score is more than 96% which means this model is very accurate and it gives us a good prediction.

reading and processing emails from TRAIN and TEST folders
Training Model using Gaussian Naibe Bayes algorithm .....
Training completed
testing trained model to predict Test Data labels
Completed classification of the Test Data .... now printing Accuracy Score by comparing the Predicted Labels with the Test Labels:
0.9615384615384616


======================= END OF PROGRAM =========================